In [12]:
import warnings
warnings.filterwarnings("ignore")

#Importing the Data
import pandas as pd
import re
import csv
from sklearn.model_selection import train_test_split
data = pd.read_csv("data/77_cancer_proteomes_CPTAC_itraq.csv",header=0,index_col=0)
clinical = pd.read_csv("data/clinical_data_breast_cancer.csv",header=0,index_col=0)
#Manipulation
clinical.columns = [c.replace(' ', '_') for c in clinical.columns]
data.columns = [c.replace(' ', '_') for c in data.columns]

clinical.rename(columns = {'Tumor--T1_Coded':'T1_Coded'}, inplace = True)
clinical.rename(columns = {'Node-Coded':'Node_Coded'}, inplace = True)
clinical.rename(columns = {'Metastasis-Coded':'Metastasis_Coded'}, inplace = True)
clinical.rename(columns = {'Integrated_Clusters_(with_PAM50)':'Integrated_Clusters_with_PAM50'}, inplace = True)
clinical.rename(columns = {'Integrated_Clusters_(no_exp)':'Integrated_Clusters_no_exp'}, inplace = True)
clinical.rename(columns = {'Integrated_Clusters_(unsup_exp)':'Integrated_Clusters_unsup_exp'}, inplace = True)

clinical.Gender[clinical.Gender == 'MALE'] = 0
clinical.Gender[clinical.Gender == 'FEMALE'] = 1

clinical.ER_Status[clinical.ER_Status == 'Positive'] = 1
clinical.ER_Status[clinical.ER_Status == 'Negative'] = 0

clinical.PR_Status[clinical.PR_Status == 'Positive'] = 1
clinical.PR_Status[clinical.PR_Status == 'Negative'] = 0

clinical.HER2_Final_Status[clinical.HER2_Final_Status == 'Positive'] = 1
clinical.HER2_Final_Status[clinical.HER2_Final_Status == 'Negative'] = 0

clinical.Tumor[clinical.Tumor == 'T1'] = 1
clinical.Tumor[clinical.Tumor == 'T2'] = 2
clinical.Tumor[clinical.Tumor == 'T3'] = 3
clinical.Tumor[clinical.Tumor == 'T4'] = 4

clinical.T1_Coded[clinical.T1_Coded == 'T1'] = 1
clinical.T1_Coded[clinical.T1_Coded == 'T_Other'] = 0

clinical.Node[clinical.Node == 'N0'] = 0
clinical.Node[clinical.Node == 'N1'] = 1
clinical.Node[clinical.Node == 'N2'] = 2
clinical.Node[clinical.Node == 'N3'] = 3

clinical.Node_Coded[clinical.Node_Coded == 'Positive'] = 1
clinical.Node_Coded[clinical.Node_Coded == 'Negative'] = 0

clinical.Metastasis[clinical.Metastasis == 'M0'] = 0
clinical.Metastasis[clinical.Metastasis == 'M1'] = 1
clinical.Metastasis_Coded[clinical.Metastasis_Coded == 'Positive'] = 1
clinical.Metastasis_Coded[clinical.Metastasis_Coded == 'Negative'] = 0

clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage I'] = 1
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IA'] = 11
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IB'] = 12
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage II'] = 2
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IIA'] = 21
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IIB'] = 22
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage III'] = 3
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IIIA'] = 31
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IIIB'] = 32
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IIIC'] = 33
clinical.AJCC_Stage[clinical.AJCC_Stage == 'Stage IV'] = 4

clinical.Converted_Stage[clinical.Converted_Stage == 'No_Conversion'] = 0
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage I'] = 1
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage IIA'] = 21
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage IIB'] = 22
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage IIIA'] = 31
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage IIIB'] = 32
clinical.Converted_Stage[clinical.Converted_Stage == 'Stage IIIC'] = 33

clinical.Survival_Data_Form[clinical.Survival_Data_Form == 'enrollment'] = 1
clinical.Survival_Data_Form[clinical.Survival_Data_Form == 'followup'] = 1

clinical.Vital_Status[clinical.Vital_Status == 'LIVING'] = 0
clinical.Vital_Status[clinical.Vital_Status == 'DECEASED'] = 1

clinical.PAM50_mRNA[clinical.PAM50_mRNA == 'Luminal A'] = 1
clinical.PAM50_mRNA[clinical.PAM50_mRNA == 'Luminal B'] = 2
clinical.PAM50_mRNA[clinical.PAM50_mRNA == 'Basal-like'] = 3
clinical.PAM50_mRNA[clinical.PAM50_mRNA == 'HER2-enriched'] = 4

clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'Basal'] = 1
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'LumA/B'] = 2
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'Her2'] = 3
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'ReacI'] = 4
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'ReacII'] = 5
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'LumA'] = 6
clinical.RPPA_Clusters[clinical.RPPA_Clusters == 'X'] = 7

#for col in clinical.columns: 
    #print(col)

In [2]:
#import pip
#pip.main(['install', 'scikit-multilearn'])

In [13]:
## Change the protein data sample names to a format matching the clinical data set
data.rename(columns=lambda x: "TCGA-%s" % (re.split('[_|-|.]',x)[0]) if bool(re.search("TCGA",x)) is True else x,inplace=True)
 
## Transpose data for the clustering algorithm since we want to divide patient samples, not proteins
data = data.transpose()
 
## Drop clinical entries for samples not in our protein data set
clinical = clinical.loc[[x for x in clinical.index.tolist() if x in data.index],:]
 
## Add clinical meta data to our protein data set, note: all numerical features for analysis start with NP_ or XP_
merged = data.merge(clinical,left_index=True,right_index=True)

print(merged.shape)

(80, 12582)


In [41]:
#Lots of NaN values so drop columns instead of rows because otherwise we would end up with 0 rows
merged = merged.dropna(axis='columns')
merged.shape

(79, 8045)

In [43]:
#Subsetting the equivocal data and looking at variable counts once again
merged = merged[merged["HER2_Final_Status"] != "Equivocal"]
merged.groupby(["ER_Status", "PR_Status", "HER2_Final_Status"]).size()

ER_Status  PR_Status  HER2_Final_Status
0          0          0                    19
                      1                     7
1          0          0                     6
                      1                     4
           1          0                    35
                      1                     8
dtype: int64

In [67]:
y_col = ["ER_Status","PR_Status","HER2_Final_Status"]
y = merged[y_col]
X = merged.drop(y_col, axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

x_train.shape


(63, 8042)

In [42]:
merged.isnull().values.any()

False

In [86]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics import accuracy_score

#Find optimal k
k_values = list(range(1,50))
k_accuracy = []

x_train = x_train.astype(float).round(6)
y_train = y_train.astype(float).round(6)
x_test = x_test.astype(float).round(6)
y_test = y_test.astype(float).round(6)

# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()

for i in k_values:
    n_neighbors = i
    classifier_new = MLkNN(n_neighbors)
    classifier_new.fit(x_train, y_train)
    predictions_new = classifier_new.predict(x_test)
    score = accuracy_score(y_test,predictions_new)
    k_accuracy.append(score)

print(k_accuracy)

[0.375, 0.4375, 0.375, 0.4375, 0.3125, 0.5, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.3125, 0.3125, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.3125, 0.375, 0.375, 0.375, 0.3125, 0.3125, 0.3125, 0.3125, 0.3125, 0.3125, 0.3125, 0.3125, 0.3125, 0.375, 0.3125, 0.3125, 0.3125, 0.3125, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.375]
